In [1]:
import matplotlib.pyplot as plt
import numpy as np
from random import random, shuffle

from scipy.integrate import simps
import pandas as pd
import sys
import math
from mpl_toolkits import mplot3d
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from scipy import interpolate
import seaborn as sns

sys.path.append("..")
from utils.load_las_data import normalize_z_with_minz_in_a_radius

from config import args
from utils.load_las_data import *
from utils.useful_functions import *
import os
import numpy as np; np. set_printoptions(suppress=True);  # display values without scientific notation
import pandas as pd
from laspy.file import File
from sklearn.neighbors import NearestNeighbors
import warnings
import random

Dataset folder in use: /home/CGaydon/Documents/LIDAR PAC/plot_vegetation_coverage/data/
Arguments were imported in DEV mode


In [2]:
def load_and_clean_single_las(las_filename):
    """Load a LAD file into a np.array, convert coordinates to meters, clean a few anomalies in plots."""
    # Parse LAS files
    las = File(las_filename, mode="r")
    x_las = las.X / 100  # we divide by 100 as all the values in las are in cm
    y_las = las.Y / 100
    z_las = las.Z / 100
    r = las.Red
    g = las.Green
    b = las.Blue
    nir = las.nir
    intensity = las.intensity
    return_num = las.return_num
    num_returns = las.num_returns
    points_nparray = np.asarray(
        [x_las, y_las, z_las, r, g, b, nir, intensity, return_num, num_returns]
    ).T

    # There is a file with 2 points 60m above others (maybe birds), we delete these points
    if las_filename.endswith("Releve_Lidar_F70.las"):
        points_nparray = points_nparray[points_nparray[:, 2] < 640]
    # We do the same for the intensity
    if las_filename.endswith("POINT_OBS8.las"):
        points_nparray = points_nparray[points_nparray[:, -2] < 32768]
    if las_filename.endswith("Releve_Lidar_F39.las"):
        points_nparray = points_nparray[points_nparray[:, -2] < 20000]

    # get the center of a rectangle bounding the points
    xy_centers = [
        (x_las.max() - x_las.min()) / 2.0,
        (y_las.max() - y_las.min()) / 2.0,
    ]
    return points_nparray, xy_centers

In [3]:
def load_my_data(args):

    # We open las files and create a training dataset
    nparray_clouds_dict = {}  # dict to store numpy array with each plot separately
    xy_centers_dict = (
        {}
    )  # we keep track of plots means to reverse the normalisation in the future

    # We iterate through las files and transform them to np array
    las_filenames = get_files_of_type_in_folder(args.las_placettes_folder_path, ".las")


    all_points_nparray = np.empty((0, len(args.input_feats) - 1))
    for las_filename in las_filenames:
        # Parse LAS files
        points_nparray, xy_centers = load_and_clean_single_las(las_filename)
#         points_nparray = transform_features_of_plot_cloud(
#             points_nparray, args
#         )
        all_points_nparray = np.append(all_points_nparray, points_nparray, axis=0)
        plot_name = get_filename_no_extension(las_filename)
        nparray_clouds_dict[plot_name] = points_nparray
        xy_centers_dict[plot_name] = xy_centers

    return all_points_nparray, nparray_clouds_dict, xy_centers_dict

In [4]:
def select_my_clouds():
    # F49 = nparray_clouds_dict["Releve_Lidar_F49"][:,:3]
    # F68 = nparray_clouds_dict["Releve_Lidar_F68"][:,:3]  # Vm everywhere
    # OBS_2021_6 = nparray_clouds_dict["2021_POINT_OBS6"][:,:3]  # "hyper flag en termes de MNT"
    # F20 = nparray_clouds_dict["Releve_Lidar_F20"][:,:3]  # "hyper flag en termes de MNT"

    plots_of_interest = [
        "Releve_Lidar_F69",
        "2021_POINT_OBS9",  # grass 100%, high slope: 0-3m to 0-0.4cm slope with 1.5
        "Releve_Lidar_F49",  # loads of high vegetation. Ok with 1.5m
        "Releve_Lidar_F68",  # dense bushes. Ok with 1.5 : adds contrast.
        "2021_POINT_OBS6",  # 75% grass (subestimation of model). might add contrats linked to grass
        "Releve_Lidar_F20",  # 75% grass, 25% mid veg. ok,
        "2021_POINT_OBS59",
    ]
    # focus on medium veg
    plots_of_interest = plots_of_interest + []
    selection = {
        key: value
        for key, value in nparray_clouds_dict.items()
        if key in plots_of_interest
    }
    return selection

In [5]:
# %matplotlib notebook
def plot_norm_impact(cloud, cloud_norm, title, output_path):
    cloud[:,2] = cloud[:,2] - np.min(cloud[:,2])
    
    fig = plt.figure(figsize=plt.figaspect(0.8) * 2.5)
    fig.patch.set_facecolor('#E0E0E0')
    fig.patch.set_alpha(0.7)
    fig.suptitle(title)

    # without norm
    ax = fig.add_subplot(2, 2, 1, projection="3d")
    ax.scatter3D(cloud[:, 0], cloud[:, 1], cloud[:, 2], c=cloud[:, 2] + 0.5)
    ax.set_xlabel("x")
    ax.set_ylabel("y")
    ax.set_zlabel("z")
    ax.view_init(elev=0., azim=0)

    ax = fig.add_subplot(2, 2, 3)
    sns.distplot(cloud[:,2],ax=ax, kde=False)


    # with norm
    cloud_norm[:,2] = cloud_norm[:,2] - np.min(cloud_norm[:,2])
    ax = fig.add_subplot(2, 2, 2, projection="3d")
    ax.scatter3D(cloud_norm[:, 0], cloud_norm[:, 1], cloud_norm[:, 2], c=cloud_norm[:, 2] + 0.5)
    ax.set_xlabel("x")
    ax.set_ylabel("y")
    ax.set_zlabel("z")
    ax.view_init(elev=0., azim=0)

    ax = fig.add_subplot(2, 2, 4)
    sns.distplot(cloud_norm[:,2],ax=ax, kde=False)

    plt.tight_layout()
    plt.savefig(output_path, dpi = 100, facecolor=fig.get_facecolor(), edgecolor='none')

### Fonctionnal code to integrate is here 

In [6]:
DIM_METERS = 20


def center_plot(cloud, xy_center):
    """Center the cloud to 0, also return the initial xymin to decenter the cloud"""
    cloud[:, :2] = cloud[:, :2] - xy_center
    return cloud


def decenter_plot(cloud, xy_center):
    cloud[:, :2] = cloud[:, :2] + xy_center
    return cloud


def xy_to_polar_coordinates(xy):
    r = np.sqrt((1.0 * xy * xy).sum(axis=1))
    teta = np.arctan2(
        xy[:, 1], xy[:, 0]
    )  # -pi, pi around (0,0) to (1,0). y and x are args in this order.
    rteta = np.stack([r, teta], axis=1)
    return rteta


def polar_coordinates_to_xy(rteta):
    x = rteta[:, 0] * np.cos(rteta[:, 1])
    y = rteta[:, 0] * np.sin(rteta[:, 1])
    xy = np.stack([x, y], axis=1)
    return xy


def create_buffer_points(cloud, ring_thickness_meters):
    """cloud is centered with xy as first coordinates in meters."""
    candidates_polar = cloud.copy()
    candidates_polar[:, :2] = xy_to_polar_coordinates(candidates_polar[:, :2])
    candidates_polar = candidates_polar[
        candidates_polar[:, 0] > (DIM_METERS // 2 - ring_thickness_meters)
    ]  # points in external ring
    candidates_polar[:, 0] = candidates_polar[:, 0] + 2 * (
        abs(DIM_METERS // 2 - candidates_polar[:, 0])
    )  # use border of plot as a mirror
    candidates_polar[:, :2] = polar_coordinates_to_xy(candidates_polar[:, :2])
    return candidates_polar


# TRUE VERSION
def normalize_z_with_smooth_spline(cloud, xy_center, pix_size, ring_thickness_meters, s=None):
    """From a regular grid, find lowest point in each cell/pixel and use them to approximate
    the DTM with a spline. Then, normalize z by flattening the ground using the DTM.
    """
    norm_cloud = cloud.copy()

    # center in order to use polar coordinate
    norm_cloud = center_plot(norm_cloud, xy_center)

    # create buffer
    buffer_points = create_buffer_points(norm_cloud, ring_thickness_meters)
    extended_cloud = np.concatenate([norm_cloud, buffer_points])

    # fit
    xy_quantified = (
        extended_cloud[:, :2] // pix_size + 0.5
    ) * pix_size  # quantify (and center) coordinates
    xy_pairs, z_argmin = npi.group_by(xy_quantified).argmin(extended_cloud[:, 2])
    sbs = SmoothBivariateSpline(
        extended_cloud[z_argmin, 0],
        extended_cloud[z_argmin, 1],
        extended_cloud[z_argmin, 2],
        kx=3,
        ky=3,
        s=s,
    )

    # predict on normcloud
    norm_cloud[:, 2] = norm_cloud[:, 2] - sbs(
        norm_cloud[:, 0], norm_cloud[:, 1], grid=False
    )
    # get back to initial coordinate system
    norm_cloud = decenter_plot(norm_cloud, xy_center)
    return norm_cloud

In [ ]:
from scipy.interpolate import SmoothBivariateSpline
import numpy_indexed as npi

np.random.seed(0)
xy_center = np.array([500,500])
# fake data
xy = (
    np.random.random((1000, 2)) * DIM_METERS + xy_center
)  # 20*20 and with an offset that needs to be removed


def fxy(x, y):
    return -np.sin(y * 2) + np.cos(x * 2) * 2 + 2 * np.cos(x) * np.sin(x * 2)


z = fxy(xy[:, 0:1], xy[:, 1:2])
xyz = np.append(xy, z, axis=1)


normalize_z_with_smooth_spline(xyz, xy_center, 1.5, 8, s=None)

In [ ]:
np.concatenate([xyz, xyz]).shape

# Try with real data

In [ ]:
rng

In [ ]:
print(len(selection))

In [ ]:
%matplotlib notebook
# range_pix_size = np.arange(0.5, 3, 0.25)
range_pix_size = [1.25]

for name, my_cloud in selection.items():
    for pix_size in range_pix_size:
        
        # here approximate but should be real coordinates !
        xy_min = my_cloud[:, :2].min(axis=0)
        xy_center = xy_min + np.array([DIM_METERS // 2, DIM_METERS // 2])
        
        s = (20 // pix_size) ** 2 * 2
        cloud = my_cloud.copy()
        output_path = (
            f"./temp_cloud_extension/extended_spline_pl_{name}_pix_size_{pix_size:.02f}.png"
        )
        cloud_norm = normalize_z_with_smooth_spline(cloud, xy_center, pix_size, 2, s=s)
        title = (
            f"Without and with normalization spline pl_{name} pix_size {pix_size:.02f} s_{s}"
        )
        print(output_path)
        plot_norm_impact(my_cloud, cloud_norm, title, output_path)

In [ ]:
cloud_norm = normalize_z_with_smooth_spline(cloud, pix_size, 15, s=s)

In [ ]:
cloud.shape

In [ ]:
# DBEUG VERSION
# def normalize_z_with_smooth_spline(cloud, pix_size, ring_thickness_meters, s=None):
name = "Releve_Lidar_F68"
cloud = nparray_clouds_dict[name]
pix_size=1.5 
ring_thickness_meters=1.5 
s=None
"""From a regular grid, find lowest point in each cell/pixel and use them to approximate
the DTM with a spline. Then, normalize z by flattening the ground using the DTM.
"""

# here approximate but should be real coordinates !
xy_min = cloud[:, :2].min(axis=0)
xy_center = xy_min + np.array([DIM_METERS // 2, DIM_METERS // 2])

norm_cloud = cloud.copy()




print(cloud.shape)
# center in order to use polar coordinate
norm_cloud = center_plot(norm_cloud, xy_center)
print(norm_cloud.shape)


# create buffer
buffer_points = create_buffer_points(norm_cloud, ring_thickness_meters)
extended_cloud = np.concatenate([norm_cloud, buffer_points])
print(buffer_points.shape, extended_cloud.shape)

# fit
xy_quantified = (
    extended_cloud[:, :2] // pix_size + 0.5
) * pix_size  # quantify (and center) coordinates
print(xy_quantified)

xy_pairs, z_argmin = npi.group_by(xy_quantified).argmin(extended_cloud[:, 2])
sbs = SmoothBivariateSpline(
    extended_cloud[z_argmin, 0],
    extended_cloud[z_argmin, 1],
    extended_cloud[z_argmin, 2],
    kx=5,
    ky=5,
    s=s,
)

norm_cloud = buffer_points.copy()

# DEBUG: Plot the approximation
norm_cloud[:, 2] = sbs(norm_cloud[:, 0], norm_cloud[:, 1], grid=False)
# norm_cloud = decenter_plot(norm_cloud, xy_center)

output_path = (
    f"./temp_cloud_extension/extended_spline_pl_{name}_pix_size_{pix_size:.02f}.png"
)
title = (
    f"Without and with normalization spline pl_{name} pix_size {pix_size:.02f} s_{s}"
)

plot_norm_impact(cloud, norm_cloud, title, output_path)

In [ ]:
norm_cloud[:,1].max() - norm_cloud[:,1].min(), (norm_cloud[:,0].max() - norm_cloud[:,0].min())

In [ ]:
def my_imshow(gr):
    plt.figure()
    plt.imshow(gr[:,:2])
    plt.colorbar()
my_imshow(buffer_points)

In [ ]:
candidates_polar = np.array([[-1.,2]])
candidates_polar[:, :2] = xy_to_polar_coordinates(candidates_polar[:, :2])
candidates_polar

In [ ]:
%matplotlib notebook
def visualize_spline_approx(cloud, title):
        
    # here approximate but should be real coordinates !
    xy_min = my_cloud[:, :2].min(axis=0)
    xy_center = xy_min + np.array([DIM_METERS // 2, DIM_METERS // 2])
    norm_cloud = normalize_z_with_smooth_spline(cloud,xy_center, 1.5, 2, s=None)
    cloud_dtm = norm_cloud.copy()
    cloud_dtm[:,2] = cloud[:,2] - cloud_dtm[:,2]

    # Visualize
    plt.figure(figsize=(15,15))
    ax = plt.axes(projection='3d')
    ax.plot_trisurf(cloud_dtm[:,0], cloud_dtm[:,1], cloud_dtm[:,2]);
    ax.scatter3D(cloud[:,0], cloud[:,1], cloud[:,2], c= "b", cmap='Greens');
    ax.view_init(elev=0., azim=0)
#     ax.set_zlim(-5, 5)
    plt.title(title)
    plt.show()


for key in selection:
    visualize_spline_approx(nparray_clouds_dict[key], key)
pass